**Integrantes: Erich Gonzalez, Daniel Gutierrez San José y Georgelys Marcano**

Enunciado: 

    - Calcular mediante paralelización con Spark la media y varianza del dataset data_ok.csv utilizando exclusivamente funciones básicas map/reduce (textFile, reduce, reduceByKey, map, flatmap, filter, count).

    - Calcular inicialmente para una sola columna y más tarde para todas las columnas del dataset.

    - Verificar que la solución propuesta es correcta con las funciones rdd.mean() y rdd.stdev().

Se deben realizar 3 versiones:

    a1: Calcula la media y varianza para la columna 4

    a2: Utilizando las operaciones de vectorización de python y arrays de numpy, utilizar la misma estructura de código de la version 1 para calcular las medias y varianzas de todas las columnas

    a3: Transforma cada celda del dataset en un elemento (j,v), donde "j" es la columna de la celda y "v" es el valor de la celda del rdd. Resuelve el problema con esta nueva estructura del dataset

### Versión a3: Transforma cada celda del dataset en un elemento (j,v), donde "j" es la columna de la celda y "v" es el valor de la celda del rdd. Resuelve el problema con esta nueva estructura del dataset

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
import numpy as np

In [ ]:
sc = SparkContext("local[*]", "v.a3-media-varianza")

In [3]:
#ruta del csv
path = "./data_ok.csv" 

#se ingresa la ruta en file del sc (rdd)
rdd = sc.textFile(path)

In [ ]:
#Se obtiene el rdd1, transformando el dataset en un array de arrays, donde cada valor es un row
rdd1 = rdd.map(lambda linea:linea.split())

# transformar a float (rdd2)
rdd2 = rdd1.map(lambda row: [float(celda) for celda in row])

#Se realiza un map para obtener una matriz con valores (j,v)
rdd3 = rdd2.map(lambda row:[(columna,valor) for columna,valor in enumerate(row)])

#Se realiza un reduce para obtener la suma de todos los valores de la matriz
sum_elements = rdd3.reduce(lambda row1,row2:[(indice,item[1]+row2[indice][1]) for indice,item in enumerate(row1)])

#Se totaliza la cantidad de elementos, para obtener un array de medias
total_item_rows = rdd2.count()
means_col = []
for index,tupla in enumerate(sum_elements):
    means = tupla[1] / total_item_rows
    means_col.append(means)
#print("Media de cada columna:", means_col[:3])

# Calcular la varianza
sum_dif_quarts = tuplas.map(lambda row: [(index, (valor[1] - means_col[index])**2) for index, valor in enumerate(row)])

# Reducir para sumar las diferencias cuadradas
sum_variance = sum_dif_quarts.reduce(lambda row1, row2: [(index, item[1] + row2[index][1]) for index, item in enumerate(row1)])

# Calcular la varianza dividiendo entre el total de filas
variance_col = [tupla[1] / total_item_rows for tupla in sum_variance]
#print(variance_col)